# Example 2

## Anomaly detection in ECG signal with AISLEX and Linear Neural Unit
In this example we demonstrate the capabilities of anomly detection with the use of AISLEX and linear neural unit on public [real ECG](https://www.physionet.org/content/cudb/1.0.0/) [1] data.

[1] F. M. Nolle and R. W. Bowser, “Creighton University Ventricular Tachyarrhythmia Database.” physionet.org, 1992. doi: 10.13026/C2X59M.

In [ ]:
Data > Ventricual Tachycardia